# Extract Lemmatization from ORACC JSON: Basic Parser
The code in this notebook will parse [ORACC](http://oracc.org) `JSON` files to extract lemmatization data for one or more projects. The resulting `csv` (Comma Separated Values) file is named `parsed.csv` and has two fields: a Text ID (e.g. `dcclt/Q000039`) and a string of lemmas in the format `lugal[king]N` (or `šarru[king]N` for Akkadian texts).

The output of the Basic Parser contains *only* text IDs and lemmas. This format is useful for so-called [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) techniques such as word clouds or [topic modeling](https://en.wikipedia.org/wiki/Topic_model). The `JSON` files, however, contain a wealth of other data, including language (Sumerian, Akkadian, Emesal, etc.), orthographic form, morphology (currently only for Sumerian and Emesal), line numbers, breakage, etc. The extended JSON parser notebook, building upon the techniques demonstrated here, will show how to extract any type of data.

In [1]:
import pandas as pd
import zipfile
import json
import tqdm
import requests
import errno
import os

## 0 Create Directories, if Necessary
The two directories needed for this script are `jsonzip` and `output`. If they do not exist they are created, else: do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [2]:
directories = ['jsonzip', 'output']
for d in directories:
    try:
        os.mkdir(d)
    except OSError as exc:
        if exc.errno !=errno.EEXIST:
            raise
        pass

## 1.1 Input Project Names
Provide a list of one or more project names, separated by commas. Note that subprojects must be listed separately, they are not included in the main project. For instance:

`saao/saa01,saao/saa02,blms`

In [3]:
projects = input('Project(s): ').lower()

Project(s): epsd2/admin/u3adm


## 1.2 Split the List of Projects
Split the list of projects and create a list of project names.

In [4]:
p = projects.split(',')               # split at each comma and make a list called `p`
p = [x.strip() for x in p]        # strip spaces left and right of each entry in `p`

## 1.3 Download the ZIP files
For each project in the list download all the `json` files from `http://build-oracc.museum.upenn.edu/json/`. The file is called `PROJECT.zip` (for instance: `dcclt.zip`). For subprojects the file is called `PROJECT-SUBPROJECT.zip` (for instance `cams-gkab.zip`). 

For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `zip` file may be 25Mb or more. Downloading may take some time and it may be advisable to chunk the downloading process. The `iter_content()` function in the `requests` library takes care of that.

If you have downloaded the files by hand (and put them in the `jsonzip` directory) you may skip this cell and jump directly to section [2.1 The Parsejson() function](#head21).

In [5]:
CHUNK = 16 * 1024
for project in p:
    project = project.replace('/', '-')
    url = "http://build-oracc.museum.upenn.edu/json/" + project + ".zip"
    file = 'jsonzip/' + project + '.zip'
    with requests.get(url, stream=True) as r:
        if r.status_code == 200:
            print("Downloading " + url + " saving as " + file)
            with open(file, 'wb') as f:
                for c in tqdm.tqdm(r.iter_content(chunk_size=CHUNK)):
                    f.write(c)
        else:
            print(url + " does not exist.")

28491it [00:52, 545.49it/s]


## <a name="head21"></a>2.1 The `parsejson()` function
The `parsejson()` function will "dig into" the `json` file (transformed into a dictionary) until it finds the relevant data. The `json` file consists of a hierarchy of `cdl` nodes; only the lowest nodes contain lemmatization data. The function goes down this hierarchy by calling itself when another `cdl` node is encountered. For more information about the data hierarchy in the [ORACC](http://oracc.org) `json` files, see [ORACC Open Data](http://oracc.org/doc/opendata/index.html).

The argument of the `parsejson()` function is a `JSON` object, essentially a Python dictionary that initially contains the entire contents of the original JSON file. The code takes the key `cdl` the value of which is a list of `JSON` dictionaries. Iterating through these dictionaries, if a dictionary contains another `cdl` node, the function calls itself with this lower-level dictionary as argument. This way the function digs deeper and deeper into the `JSON` tree, until it does not encounter a `cdl` key anymore. Here we are at the level of individual words. The code checks for a key `f`, if it exists the value of that key (another dictionary) is appended to the list `lemm_l`. The list `lemm_l`, which is initiated outside of the function proper, will become a list of dictionaries, where each dictionary represents a single word.

The variable `id_text` consists of a project abbreviation, such as `blms` or `cams/gkab` plus a text ID, in the format `cams/gkab/P338616` or `dcclt/Q000039`. The `id_text` is a global variable that is defined each time `parsejson()` is called in the main process. Therefore, it can be accessed from within the function and is added to the lemmatization data of every word.

In [6]:
def parsejson(text):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            parsejson(JSONobject)
        if "f" in JSONobject:
            lemm = JSONobject["f"]
            lemm["id_text"] = id_text
            lemm_l.append(lemm)
    return

## 2.2 Call the `parsejson()` function for every `JSON` file
The code in this cell will iterate through the list of projects entered above (1.1). For each project the `JSON` zip file is located in the directory `jsonzip`, named PROJECT.zip. The `zip` file contains a directory that is called `corpusjson` that contains a JSON file for every text that is available in that corpus. The files are called after their text IDs in the pattern `P######.json` (or `Q######.json` or `X######.json`).

The function `namelist()` of the `zipfile` package is used to create a list of the names of all the files in the ZIP. From this list we select all the file names in the `corpusjson` directory with extension `.json` (this way we exclude the name of the directory itself). 

Each of these files is read from the `zip` file and loaded with the command `json.loads()`, which transforms the string into a proper JSON object. 

This JSON object (essentially a Python dictionary), which is called `data_json` is now sent to the `parsejson()` function. The function adds lemmata to the `lemm_l` list. In the end, `lemm_l` will contain as many list elements as there are words in all the texts in the projects requested.

In [7]:
lemm_l = [] # initiate the list that will hold all the lemmatization data of all texts in all requested projects
for project in p:
    print("Parsing " + project)
    file = "jsonzip/" + project.replace("/", "-") + ".zip"
    try:
        z = zipfile.ZipFile(file)       # create a Zipfile object
    except:
        print(file + " does not exist or is not a proper ZIP file")
        continue
    files = z.namelist()     # list of all the files in the ZIP
    files = [name for name in files if "corpusjson" in name and name[-5:] == '.json']                                                                                                  #that holds all the P, Q, and X numbers.
    for filename in tqdm.tqdm(files):                            #iterate over the file names
        id_text = project + filename[-13:-5] # id_text is, for instance, blms/P414332
        try:
            text = z.read(filename).decode('utf-8')         #read and decode the json file of one particular text
            data_json = json.loads(text)                # make it into a json object (essentially a dictionary)
            parsejson(data_json)               # and send to the parsejson() function
        except:
            print(id_text + ' is not available or not complete')
    z.close()

Parsing epsd2/admin/u3adm


100%|███████████████████████████████████| 71239/71239 [03:42<00:00, 320.68it/s]


## 3 Data Structuring
### 3.1 Transform the Data into a DataFrame
The list `lemm_l` is transformed into a `pandas` dataframe (`word_df`) for further manipulation. The `pandas` function `fillna('')` fills gaps in the dataframe. If a particular data field is not available for a particular row, `pandas` will fill that cell with NaN ("Not a Number"). The data type of NaN is numeric, which creates problems in string manipulation further down the road (for instance in creating a Lemma column, section 3.2). The function `fillna()` will fill all empty cells with the argument it is given - in this case the empty string.

For various reasons not all JSON files will have all data types that potentially exist in an [ORACC](http://oracc.org) lemmatization. For instance, only Sumerian words have a `base`, so if your data set has no Sumerian, this column will not exist in the DataFrame. Where such fields are referenced below, the code may fail and you may need to adjust some lines.

In [8]:
word_df = pd.DataFrame(lemm_l)
word_df = word_df.fillna('')      # replace NaN (Not a Number) with empty string
word_df

,base,cf,cont,delim,epos,form,gdl,gw,id_text,lang,morph,norm,norm0,pos,sense,stem
0,,,,,,3(diš),"[{'n': 'n', 'form': '3(diš)', 'id': 'P124149.3...",,epsd2/admin/u3adm/P124149,sux,,,,n,,
1,udu,udu,,,N,udu,"[{'v': 'udu', 'id': 'P124149.3.2.0'}]",sheep,epsd2/admin/u3adm/P124149,sux,~,,udu,N,sheep,
2,,,,,,6(diš),"[{'n': 'n', 'form': '6(diš)', 'id': 'P124149.4...",,epsd2/admin/u3adm/P124149,sux,,,,n,,
3,u₈,u,,,N,u₈,"[{'v': 'u₈', 'id': 'P124149.4.2.0'}]",ewe,epsd2/admin/u3adm/P124149,sux,~,,u,N,"sheep, ewe",
4,,,,,,2(diš),"[{'n': 'n', 'form': '2(diš)', 'id': 'P124149.5...",,epsd2/admin/u3adm/P124149,sux,,,,n,,
5,maš₂-gal,mašgal,,,N,maš₂-gal,"[{'v': 'maš₂', 'id': 'P124149.5.2.0', 'delim':...",goat,epsd2/admin/u3adm/P124149,sux,~,,mašgal,N,billy goat,
6,šu-gid₂,šugid,,,N,šu-gid₂,"[{'v': 'šu', 'id': 'P124149.6.1.0', 'delim': '...",~animal,epsd2/admin/u3adm/P124149,sux,~,,šugid,N,"designation of animals, unfit for work?",
7,e₂-muhaldim,emuhaldim,,,N,e₂-muhaldim,"[{'v': 'e₂', 'id': 'P124149.6.2.0', 'delim': '...",kitchen,epsd2/admin/u3adm/P124149,sux,~,,emuhaldim,N,kitchen,
8,mu,mu,,,N,mu,"[{'v': 'mu', 'id': 'P124149.7.1.0'}]",name,epsd2/admin/u3adm/P124149,sux,~,,mu,N,name,
9,gar₃-du,gardu,,,N,gar₃-du-e-ne-še₃,"[{'v': 'gar₃', 'id': 'P124149.7.2.0', 'delim':...",soldier,epsd2/admin/u3adm/P124149,sux,"~,ene.eše",,"gardu,ene.eše",N,soldier,


## 3.1 Remove Spaces and Commas from Guide Word and Sense
Spaces and commas in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens and nothing (empty string), respectively. By default the `replace()` function in `pandas` will match the entire string (that is, "lugal" matches "lugal" but there is no match between "l" and "lugal"). In order to match partial strings the parameter `regex` must be set to `True`.

The `replace()` function takes a nested dictionary as argument. The top-level keys identify the columns on which the `replace()` function should operate (in this case 'gw' and 'sense'). The value of each key is another dictionary with the search string as key and the replace string as value.

In [10]:
findreplace = {' ' : '-', ',' : ''}
word_df = word_df.replace({'gw' : findreplace, 'sense' : findreplace}, regex = True)

## 3.2 Create a `lemma` column
A lemma, [ORACC](http://oracc.org) style, combines Citation Form, GuideWord and POS into a unique reference to one particular lemma in a standard dictionary, as in `lugal[king]N` (Sumerian) or `šarru[king]N`. Usually, not all words in a text are lemmatized, because a word may be (partly) broken and/or unknown. Unlemmatized and unlemmatizable words will receive a place-holder lemmatization that consists of the transliteration of the word (instead of the Citation Form), with `NA` as GuideWord and `NA` as POS, as in `i-bu-x[NA]NA`. Note that `NA` is a string. 

In [11]:
word_df["lemma"] = word_df["cf"] + '[' + word_df["gw"] + ']' + word_df["pos"]
word_df.loc[word_df["cf"] == "" , 'lemma'] = word_df['form'] + '[NA]NA'
word_df[['id_text', 'lemma']]

,id_text,lemma
0,epsd2/admin/u3adm/P124149,3(diš)[NA]NA
1,epsd2/admin/u3adm/P124149,udu[sheep]N
2,epsd2/admin/u3adm/P124149,6(diš)[NA]NA
3,epsd2/admin/u3adm/P124149,u[ewe]N
4,epsd2/admin/u3adm/P124149,2(diš)[NA]NA
5,epsd2/admin/u3adm/P124149,mašgal[goat]N
6,epsd2/admin/u3adm/P124149,šugid[~animal]N
7,epsd2/admin/u3adm/P124149,emuhaldim[kitchen]N
8,epsd2/admin/u3adm/P124149,mu[name]N
9,epsd2/admin/u3adm/P124149,gardu[soldier]N


## 3.3 Group by Textid
Get all the lemmas that belong to a single text in one row (one row = one document). The `agg()` (aggregate) function, which works on the result of a `groupby()` process aggregates columns of the original dataframe. The aggregate function takes a dictionary in which the keys are column names and the values are functions to be used in the aggregation process. The example below has only one such function (`' '.join` will join all entries in the colum `lemma` with a space in between); one may specify (the same or different) functions for different columns, for instance:
```python
word_df = word_df.groupby("textid").agg({"lemma": ' '.join, "base": ' '.join})
```
The process will create a compound index. It is useful to flatten the index with a reset (`reset_index()`).

In [12]:
doc_df = word_df.groupby("id_text").agg({"lemma": ' '.join})
doc_df = doc_df.reset_index()
doc_df

,id_text,lemma
0,epsd2/admin/u3adm/P100001,a[arm]N ud[sun]N x[NA]NA 5(u)[NA]NA 4(diš)[NA]...
1,epsd2/admin/u3adm/P100002,1(geš₂)[NA]NA 3(u)[NA]NA 6(diš)[NA]NA 1/2(diš)...
2,epsd2/admin/u3adm/P100003,1(diš)[NA]NA geme[worker]N ud[sun]N 1(diš)-še₃...
3,epsd2/admin/u3adm/P100004,2(diš)[NA]NA geme[worker]N ki[place]N lugal-si...
4,epsd2/admin/u3adm/P100005,1(barig)[NA]NA lu₂-sag₁₀[NA]NA dumu[child]N ur...
5,epsd2/admin/u3adm/P100006,5(diš)[NA]NA ŋuruš[male]N x[NA]NA 1(diš)[NA]NA...
6,epsd2/admin/u3adm/P100007,4(geš₂)[NA]NA šanabi[two-thirds]NU sar[garden]...
7,epsd2/admin/u3adm/P100008,1(diš)[NA]NA ŋuruš[male]N nagar[carpenter]N ud...
8,epsd2/admin/u3adm/P100009,5(geš₂)[NA]NA 3(u)[NA]NA 4(diš)[NA]NA ŋuruš[ma...
9,epsd2/admin/u3adm/P100010,3(u)[NA]NA 4(diš)[NA]NA ŋuruš[male]N ud[sun]N ...


# 4 Save the Results
## 4.1 Save Results in CSV file
The output file is called `parsed.csv` and is placed in the directory `output`. In most cases, `csv` files open automatically in Excel. This program does not deal well with `utf-8` encoding. If you intend to use the file in Excel, change `encoding ='utf-8'` to `encoding='utf-16'`. For usage in computational text analysis applications `utf-8` is usually preferred. The option `index=False` excludes the (numerical) index from the saved `csv`.

In [ ]:
savefile =  'parsed.csv'
with open('output/' + savefile, 'w', encoding="utf-8") as w:
    doc_df.to_csv(w, index=False)

## 4.2 Pickle
The Pandas function `to_pickle()` writes a binary file that can be opened in a later phase of the project with the `read_pickle()` command and will reproduce exactly the same DataFrame. The resulting file cannot be used in other programs but preserves the data structure of the DataFrame.

In [ ]:
pickled = "parsed.p"
with open('output/' + pickled, 'wb') as w:
    doc_df.to_pickle(w)